بسم الله الرحمن الرحیم

In [ ]:
print("Mohammad Reza Hasanabadi")

Mohammad Reza Hasanabadi


## In the following codes: you may need to change the path to your dataset

In [ ]:
ls ../input/the-lj-speech-dataset/LJSpeech-1.1/wavs/*.wav | tail -n+10 > train_files.txt

In [ ]:
ls ../input/the-lj-speech-dataset/LJSpeech-1.1/wavs/*.wav | head -n10 > test_files.txt

In [ ]:
ls

In [ ]:
cat test_files.txt

## Importing classes

In [ ]:
import librosa
import math
import torch
import torch.utils.data
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from librosa.core import load
#from librosa.core import loadmel_basis
from librosa.util import normalize

from pathlib import Path
import numpy as np
import random
##
import torch.nn as nn
import torch.nn.functional as F
import torch
from librosa.filters import mel as librosa_mel_fn
from torch.nn.utils import weight_norm
import numpy as np
import scipy.io.wavfile
#########

In [ ]:
pip install librosa==0.9.1

In [ ]:
import librosa
print(librosa.__version__)

In [ ]:
#########
data_path='../input/mrh-scripts'
#
def files_to_list(filename):
    """
    Takes a text file of filenames and makes a list of filenames
    """
    with open(filename, encoding="utf-8") as f:
        files = f.readlines()

    files = [f.rstrip() for f in files]
    return files
########
filename=Path(data_path) / "train_files.txt"
audio_files=files_to_list(filename)
print(type(audio_files))
print(len(audio_files))
print('\n\n########################################\n\n')
#audio_files = [x for x in files]
#print(type(files))
#print(len(files))
#print(audio_files)
random.seed(1234)
random.shuffle(audio_files)

###################################
def save_sample(file_path, sampling_rate, audio):
    """Helper function to save sample

    Args:
        file_path (str or pathlib.Path): save file path
        sampling_rate (int): sampling rate of audio (usually 22050)
        audio (torch.FloatTensor): torch array containing audio in [-1, 1]
    """
    audio = (audio.numpy() * 32768).astype("int16")
    scipy.io.wavfile.write(file_path, sampling_rate, audio)

<class 'list'>
13091


########################################




In [ ]:
##############################################################################
class AudioDataset(torch.utils.data.Dataset):
    """
    This is the main class that calculates the spectrogram and returns the
    spectrogram, audio pair.
    """

    def __init__(self, training_files, segment_length, sampling_rate, augment=True):
        self.sampling_rate = sampling_rate
        self.segment_length = segment_length
        self.audio_files = files_to_list(training_files)
        self.audio_files = [Path(training_files).parent / x for x in self.audio_files]
        random.seed(1234)
        random.shuffle(self.audio_files)
        self.augment = augment

    def __getitem__(self, index):
        # Read audio
        filename = self.audio_files[index]
        audio, sampling_rate = self.load_wav_to_torch(filename)
        # Take segment
        if audio.size(0) >= self.segment_length:
            max_audio_start = audio.size(0) - self.segment_length
            audio_start = random.randint(0, max_audio_start)
            audio = audio[audio_start : audio_start + self.segment_length]
        else:
            audio = F.pad(
                audio, (0, self.segment_length - audio.size(0)), "constant"
            ).data

        # audio = audio / 32768.0
        return audio.unsqueeze(0)

    def __len__(self):
        return len(self.audio_files)

    def load_wav_to_torch(self, full_path):
        """
        Loads wavdata into torch array
        """
        data, sampling_rate = load(full_path, sr=self.sampling_rate)
        data = 0.95 * normalize(data)

        if self.augment:
            amplitude = np.random.uniform(low=0.3, high=1.0)
            data = data * amplitude

        return torch.from_numpy(data).float(), sampling_rate

# MRH: Original Audio2Mel :

In [ ]:
#######################################################################################################3
class Audio2Mel(nn.Module):
    def __init__(
        self,
        n_fft=1024,
        hop_length=256,
        win_length=1024,
        sampling_rate=22050,
        n_mel_channels=80,
        mel_fmin=0.0,
        mel_fmax=None,
    ):
        super().__init__()
        ##############################################
        # FFT Parameters                              #
        ##############################################
        window = torch.hann_window(win_length).float()
        mel_basis = librosa_mel_fn(
            sampling_rate, n_fft, n_mel_channels, mel_fmin, mel_fmax
        )
        mel_basis = torch.from_numpy(mel_basis).float()
        print('mel_basis.size()= ',mel_basis.size())
        self.register_buffer("mel_basis", mel_basis)
        self.register_buffer("window", window)
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.win_length = win_length
        self.sampling_rate = sampling_rate
        self.n_mel_channels = n_mel_channels

    def forward(self, audio):
        print('mrh1 = ',audio.size())
        p = (self.n_fft - self.hop_length) // 2
        audio = F.pad(audio, (p, p), "reflect").squeeze(1)
        fft = torch.stft(
            audio,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            win_length=self.win_length,
            window=self.window,
            return_complex=True,
            center=False,
        )
        print('mrh1 = ',fft.size())
        print('\nmrh2 = ',fft)
        print('\nmrh22= ',fft.size())
        real_part, imag_part = fft.unbind(-1)
        print('mrh3 = ',fft.size())
        magnitude = torch.sqrt(real_part ** 2 + imag_part ** 2)
        mel_output = torch.matmul(self.mel_basis, magnitude)
        log_mel_spec = torch.log10(torch.clamp(mel_output, min=1e-5))
        return log_mel_spec

# MRH: Modified Audio2Mel:

In [ ]:
#######################################################################################################3
class mrh_Audio2Mel(nn.Module):
    def __init__(
        self,
        n_fft=1024,
        hop_length=256,
        win_length=1024,
        sampling_rate=22050,
        n_mel_channels=80,
        mel_fmin=0.0,
        mel_fmax=None,
    ):
        super().__init__()
        ##############################################
        # FFT Parameters                              #
        ##############################################
        window = torch.hann_window(win_length).float()
        mel_basis = librosa_mel_fn(
            sampling_rate, n_fft, n_mel_channels, mel_fmin, mel_fmax
        )
        mel_basis = torch.from_numpy(mel_basis).float()
        #print('mel_basis.size()= ',mel_basis.size())
        self.register_buffer("mel_basis", mel_basis)
        self.register_buffer("window", window)
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.win_length = win_length
        self.sampling_rate = sampling_rate
        self.n_mel_channels = n_mel_channels

    def forward(self, audio):
        #print('mrh1 = ',audio.size())
        p = (self.n_fft - self.hop_length) // 2
        audio = F.pad(audio, (p, p), "reflect").squeeze(1)
        fft = torch.stft(
            audio,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            win_length=self.win_length,
            window=self.window,
            return_complex=True,
            center=False,
        )
        #print('mrh1 = ',fft.size())
        #print('\nmrh2 = ',fft)
        #print('\nmrh22= ',fft.size())
        #MRH:
        #real_part, imag_part = fft.unbind(-1)
        #magnitude = torch.sqrt(real_part ** 2 + imag_part ** 2)
        x=fft
        x_real=x.real
        x_img=x.imag
        magnitude = torch.sqrt(x_real** 2 + x_img** 2)
        #print('magnitude.size() = ',magnitude.size())

        mel_output = torch.matmul(self.mel_basis, magnitude)
        #print('mel_output.size() = ',mel_output.size())
        log_mel_spec = torch.log10(torch.clamp(mel_output, min=1e-5))
        #print('log_mel_spec.size() = ',log_mel_spec.size())
        return log_mel_spec

In [ ]:
x = torch.rand(1,5,3, dtype=torch.cfloat)
print('x= ',x)
print(x.shape)
#
#y=x.unbind(-1)
#print('\ny= ',y)
#print(len(y))
x_real=x.real
print('\nx_real= ',x_real)
#x_real_unbind=x_real.unbind(-1)
#print('\nx_real_unbind= ',x_real_unbind)
#print('\nx_real_unbind size()= ',np.size(x_real_unbind))
x_img=x.imag
print('\nx_img= ',x_img)
#x_img_unbind=x_img.unbind(-1)
#print('\nx_img_unbind= ',x_img_unbind)
#print('\nx_img_unbind size()= ',np.size(x_img_unbind))
#
#x_img_unbind**2
#x_img_unbind=list(x_img_unbind)
#x_real_unbind=list(x_real_unbind)
magnitude1 = torch.sqrt(x_real** 2 + x_img** 2)
print('magnitude1= ',magnitude1 )
#
magnitude2 = torch.sqrt(x)
print('magnitude2= ',magnitude2)

In [ ]:
n_mel_channels=80
fft = mrh_Audio2Mel(n_mel_channels=n_mel_channels).cuda()
x_t=torch.rand(1,1,88200)
x_t = x_t.cuda()
s_t = fft(x_t)#.detach()


In [ ]:
class Audio2MFCC(nn.Module):
    def __init__(
        self,
        n_fft=1024,
        hop_length=256,
        win_length=1024,
        sampling_rate=22050,
        n_mel_channels=36,
        mel_fmin=0.0,
        mel_fmax=None,
    ):
        super().__init__()
        ##############################################
        # FFT Parameters                              #
        ##############################################

        self.n_fft = n_fft
        self.hop_length = hop_length
        self.win_length = win_length
        self.sampling_rate = sampling_rate
        self.n_mel_channels = n_mel_channels

    def forward(self, x):
        y=x[0,:,:]
        #    print('x= ',x.size())
        #    print('y= ',y.size())
        z=y.cpu().numpy()
        #print(type(z))
        #print('z shape === ',z.shape)
        w=z.reshape(x.size(2),)
        #print('w shape= ',w.shape)

        #MB: 01.04.02 %2 %%%%%%%%%%%%%%%%%%%%%-------------------->>>>>>>
        mfccs = librosa.feature.mfcc(w, sr=self.sampling_rate,n_mfcc=36, n_fft=1024, hop_length=258, win_length=1024)

        #print('mfccs shape== ',mfccs.shape)
        frame_number=mfccs.shape[1]
        #print(self.n_mel_channels,frame_number)
        #print('x.size(0)= ',type(x.size(0)))
        t = torch.randn(x.size(0),self.n_mel_channels,frame_number)
        #print('t.size()== ',t.size())
        #print(x)
        ####

        ####
        for i in np.arange(x.size(0)):
            #print(i)
            y=x[i,:,:]
            #print('y= ',y.size())
            z=y.cpu().numpy()
            #print(type(z))
            #print('z shape === ',z.shape)
            w=z.reshape(x.size(2),)
            #print('w shape=== ',w.shape)
            #MB: 01.04.02 %3 %%%%n_mfcc =24%%%%%%%%%%%%%%%%%------------>>>>>>
            mfccs = librosa.feature.mfcc(w, sr=self.sampling_rate,n_mfcc=36, n_fft=1024, hop_length=258, win_length=1024)

            Q=torch.tensor(mfccs)
            #print('Q== ',Q.size())
            #print(t[i,:,:])
            t[i,:,:]=Q

            #print(t[i,:,:])
        return t
######

# MRH: Original MelGAN Model

In [ ]:
########### Model ##############

from torch.nn.utils import weight_norm

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


def WNConv1d(*args, **kwargs):
    return weight_norm(nn.Conv1d(*args, **kwargs))


def WNConvTranspose1d(*args, **kwargs):
    return weight_norm(nn.ConvTranspose1d(*args, **kwargs))
################
class ResnetBlock(nn.Module):
    def __init__(self, dim, dilation=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.LeakyReLU(0.2),
            nn.ReflectionPad1d(dilation),
            WNConv1d(dim, dim, kernel_size=3, dilation=dilation),
            nn.LeakyReLU(0.2),
            WNConv1d(dim, dim, kernel_size=1),
        )
        self.shortcut = WNConv1d(dim, dim, kernel_size=1)

    def forward(self, x):
        return self.shortcut(x) + self.block(x)
#######
####################################################################################################
class Generator(nn.Module):
    def __init__(self, input_size, ngf, n_residual_layers):
        super().__init__()
        ratios = [8, 8, 2, 2]
        self.hop_length = np.prod(ratios)
        mult = int(2 ** len(ratios))

        model = [
            nn.ReflectionPad1d(3),
            WNConv1d(input_size, mult * ngf, kernel_size=7, padding=0),
        ]
        MRH_in=32
        # Upsample to raw audio scale
        for i, r in enumerate(ratios):
            #print('mult= ',mult)
            model += [
                nn.LeakyReLU(0.2),
                WNConvTranspose1d(
                    mult * ngf,
                    mult * ngf // 2,
                    kernel_size=r * 2,
                    stride=r,
                    padding=r // 2 + r % 2,
                    output_padding=r % 2,
                ),
            ]
            #MRH
            #print('output channel= ',mult * ngf // 2)
            kernel_size=r * 2
            stride=r
            padding=r // 2 + r % 2
            output_padding=r % 2
            #MRH_dilation=1
            #MRH_length=(MRH_in-1)*stride-2*padding +MRH_dilation*(kernel_size-1)+ output_padding + 1
            #MRH_in=MRH_length
            #print(input_size)
            #print("MRH calculated output Length= ",MRH_length)

            for j in range(n_residual_layers):
                model += [ResnetBlock(mult * ngf // 2, dilation=3 ** j)]

            mult //= 2

        model += [
            nn.LeakyReLU(0.2),
            nn.ReflectionPad1d(3),
            WNConv1d(ngf, 1, kernel_size=7, padding=0),
            nn.Tanh(),
        ]

        self.model = nn.Sequential(*model)
        self.apply(weights_init)

    def forward(self, x):
        #print(x.size())
        return self.model(x)
#######
############################################################################
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.choices = nn.ModuleDict({
                'conv': nn.Conv2d(10, 10, 3),
                'pool': nn.MaxPool2d(3)
        })
        self.activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['prelu', nn.PReLU()]
        ])

    def forward(self, x, choice='conv', act='prelu'):
        x = self.choices[choice](x)
        #print(x.size())
        x = self.activations[act](x)
        #print(x.size())
        return x
#####################################################################################################
class NLayerDiscriminator(nn.Module):
    def __init__(self, ndf, n_layers, downsampling_factor):
        super().__init__()
        model = nn.ModuleDict()

        model["layer_0"] = nn.Sequential(
            nn.ReflectionPad1d(7),
            WNConv1d(1, ndf, kernel_size=15),
            nn.LeakyReLU(0.2, True),
        )

        nf = ndf
        stride = downsampling_factor
        for n in range(1, n_layers + 1):
            nf_prev = nf
            nf = min(nf * stride, 1024)

            model["layer_%d" % n] = nn.Sequential(
                WNConv1d(
                    nf_prev,
                    nf,
                    kernel_size=stride * 10 + 1,
                    stride=stride,
                    padding=stride * 5,
                    groups=nf_prev // 4,
                ),
                nn.LeakyReLU(0.2, True),
            )

        nf = min(nf * 2, 1024)
        model["layer_%d" % (n_layers + 1)] = nn.Sequential(
            WNConv1d(nf_prev, nf, kernel_size=5, stride=1, padding=2),
            nn.LeakyReLU(0.2, True),
        )

        model["layer_%d" % (n_layers + 2)] = WNConv1d(
            nf, 1, kernel_size=3, stride=1, padding=1
        )

        self.model = model

    def forward(self, x):
        results = []
        for key, layer in self.model.items():
            x = layer(x)
            results.append(x)
        return results


class Discriminator(nn.Module):
    def __init__(self, num_D, ndf, n_layers, downsampling_factor):
        super().__init__()
        self.model = nn.ModuleDict()
        for i in range(num_D):
            self.model[f"disc_{i}"] = NLayerDiscriminator(
                ndf, n_layers, downsampling_factor
            )

        self.downsample = nn.AvgPool1d(4, stride=2, padding=1, count_include_pad=False)
        self.apply(weights_init)

    def forward(self, x):
        results = []
        for key, disc in self.model.items():
            #print('key= ', key)
            #print('discrimiator= ',disc)
            results.append(disc(x))
            #print('Length results== ',len(results))
            #print('results', results)
            x = self.downsample(x)
        return results

# MRH: HyperParameters:

In [ ]:

save_path='/kaggle/working/MRH-Baseline/'
load_path=None
n_mel_channels=80
ngf=32
n_residual_layers=3
ndf=16
num_D=3
n_layers_D=4
downsamp_factor=4
lambda_feat=10
#
data_path='../input/mrh-scripts'
#
batch_size=16
seq_len=8192
epochs=3500
log_interval=100
save_interval=40650
n_test_samples=100
#####################

# MRH:  Original Mel-GAN Code

In [ ]:
############  train.py ###########

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import yaml
import numpy as np
import time
import argparse
from pathlib import Path

###

def main():
    #args = parse_args()
    save_path='/kaggle/working/MRH-Baseline/'
    root = Path(save_path)
    load_root = Path(load_path) if load_path else None
    root.mkdir(parents=True, exist_ok=True)

    ####################################
    # Dump arguments and create logger #
    ####################################
    #with open(root / "args.yml", "w") as f:
        #yaml.dump(args, f)
    writer = SummaryWriter(str(root))


    #######################
    # Load PyTorch Models #
    #######################
    netG = Generator(n_mel_channels, ngf, n_residual_layers).cuda()
    netD = Discriminator(num_D, ndf, n_layers_D, downsamp_factor).cuda()
    fft = mrh_Audio2Mel(n_mel_channels=n_mel_channels).cuda()

    #print(netG)
    #print(netD)

    #####################
    # Create optimizers #
    #####################
    optG = torch.optim.Adam(netG.parameters(), lr=1e-4, betas=(0.5, 0.9))
    optD = torch.optim.Adam(netD.parameters(), lr=1e-4, betas=(0.5, 0.9))

    if load_root and load_root.exists():
        netG.load_state_dict(torch.load(load_root / "netG.pt"))
        optG.load_state_dict(torch.load(load_root / "optG.pt"))
        netD.load_state_dict(torch.load(load_root / "netD.pt"))
        optD.load_state_dict(torch.load(load_root / "optD.pt"))

    #######################
    # Create data loaders #
    #######################
    train_set = AudioDataset("train_files.txt", seq_len, sampling_rate=22050)
    test_set = AudioDataset("test_files.txt",22050 * 4, sampling_rate=22050,augment=False)

    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=4)
    test_loader = DataLoader(test_set, batch_size=1)

    ##########################
    # Dumping original audio #
    ##########################
    test_voc = []
    test_audio = []
    for i, x_t in enumerate(test_loader):
        x_t = x_t.cuda()
        s_t = fft(x_t).detach()

        test_voc.append(s_t.cuda())
        test_audio.append(x_t)

        audio = x_t.squeeze().cpu()
        save_sample(root / ("original_%d.wav" % i), 22050, audio)
        writer.add_audio("original/sample_%d.wav" % i, audio, 0, sample_rate=22050)

        if i == n_test_samples - 1:
            break

    costs = []
    start = time.time()

    # enable cudnn autotuner to speed up training
    torch.backends.cudnn.benchmark = True

    best_mel_reconst = 1000000
    steps = 0
    for epoch in range(1, epochs + 1):
        for iterno, x_t in enumerate(train_loader):
            x_t = x_t.cuda()
            s_t = fft(x_t).detach()
            x_pred_t = netG(s_t.cuda())

            with torch.no_grad():
                s_pred_t = fft(x_pred_t.detach())
                s_error = F.l1_loss(s_t, s_pred_t).item()

            #######################
            # Train Discriminator #
            #######################
            D_fake_det = netD(x_pred_t.cuda().detach())
            D_real = netD(x_t.cuda())

            loss_D = 0
            for scale in D_fake_det:
                loss_D += F.relu(1 + scale[-1]).mean()

            for scale in D_real:
                loss_D += F.relu(1 - scale[-1]).mean()

            netD.zero_grad()
            loss_D.backward()
            optD.step()

            ###################
            # Train Generator #
            ###################
            D_fake = netD(x_pred_t.cuda())

            loss_G = 0
            for scale in D_fake:
                loss_G += -scale[-1].mean()

            loss_feat = 0
            feat_weights = 4.0 / (n_layers_D + 1)
            D_weights = 1.0 / num_D
            wt = D_weights * feat_weights
            for i in range(num_D):
                for j in range(len(D_fake[i]) - 1):
                    loss_feat += wt * F.l1_loss(D_fake[i][j], D_real[i][j].detach())

            netG.zero_grad()
            (loss_G + lambda_feat * loss_feat).backward()
            optG.step()

            ######################
            # Update tensorboard #
            ######################
            costs.append([loss_D.item(), loss_G.item(), loss_feat.item(), s_error])

            writer.add_scalar("loss/discriminator", costs[-1][0], steps)
            writer.add_scalar("loss/generator", costs[-1][1], steps)
            writer.add_scalar("loss/feature_matching", costs[-1][2], steps)
            writer.add_scalar("loss/mel_reconstruction", costs[-1][3], steps)
            steps += 1

            if steps % save_interval == 0:
                st = time.time()
                with torch.no_grad():
                    for i, (voc, _) in enumerate(zip(test_voc, test_audio)):
                        pred_audio = netG(voc)
                        pred_audio = pred_audio.squeeze().cpu()
                        save_sample(root / ("generated_%d.wav" % i), 22050, pred_audio)
                        writer.add_audio(
                            "generated/sample_%d.wav" % i,
                            pred_audio,
                            epoch,
                            sample_rate=22050,
                        )

                torch.save(netG.state_dict(), root / "netG.pt")
                torch.save(optG.state_dict(), root / "optG.pt")

                torch.save(netD.state_dict(), root / "netD.pt")
                torch.save(optD.state_dict(), root / "optD.pt")

                if np.asarray(costs).mean(0)[-1] < best_mel_reconst:
                    best_mel_reconst = np.asarray(costs).mean(0)[-1]
                    torch.save(netD.state_dict(), root / "best_netD.pt")
                    torch.save(netG.state_dict(), root / "best_netG.pt")

                print("Took %5.4fs to generate samples" % (time.time() - st))
                print("-" * 100)

            if steps % log_interval == 0:
                print(
                    "Epoch {} | Iters {} / {} | ms/batch {:5.2f} | loss {}".format(
                        epoch,
                        iterno,
                        len(train_loader),
                        1000 * (time.time() - start) / log_interval,
                        np.asarray(costs).mean(0),
                    )
                )
                costs = []
                start = time.time()


if __name__ == "__main__":

    main()

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

## MRH:  In case of Using multiple GPUs:  using "device" instead of "cuda"

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import yaml
import numpy as np
import time
import argparse
from pathlib import Path

###

def main():
    #args = parse_args()
    save_path='/kaggle/working/MRH-Baseline/'
    root = Path(save_path)
    load_root = Path(load_path) if load_path else None
    root.mkdir(parents=True, exist_ok=True)

    ####################################
    # Dump arguments and create logger #
    ####################################
    #with open(root / "args.yml", "w") as f:
        #yaml.dump(args, f)
    writer = SummaryWriter(str(root))

    ##### MRH: Multiple GPUs
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    #######################
    # Load PyTorch Models #
    #######################
    # MR:
    #netG = Generator(n_mel_channels, ngf, n_residual_layers).cuda()
    #netD = Discriminator(num_D, ndf, n_layers_D, downsamp_factor).cuda()
    #fft = mrh_Audio2Mel(n_mel_channels=n_mel_channels).cuda()

    netG = Generator(n_mel_channels, ngf, n_residual_layers)
    netG= nn.DataParallel(netG)
    netG.to(device)

    netD = Discriminator(num_D, ndf, n_layers_D, downsamp_factor)
    netD= nn.DataParallel(netD)
    netD.to(device)

    fft = mrh_Audio2Mel(n_mel_channels=n_mel_channels)
    fft= nn.DataParallel(fft)
    fft.to(device)

    #print(netG)
    #print(netD)

    #####################
    # Create optimizers #
    #####################
    optG = torch.optim.Adam(netG.parameters(), lr=1e-4, betas=(0.5, 0.9))
    optD = torch.optim.Adam(netD.parameters(), lr=1e-4, betas=(0.5, 0.9))

    if load_root and load_root.exists():
        netG.load_state_dict(torch.load(load_root / "netG.pt"))
        optG.load_state_dict(torch.load(load_root / "optG.pt"))
        netD.load_state_dict(torch.load(load_root / "netD.pt"))
        optD.load_state_dict(torch.load(load_root / "optD.pt"))

    #######################
    # Create data loaders #
    #######################
    train_set = AudioDataset("train_files.txt", seq_len, sampling_rate=22050)
    test_set = AudioDataset("test_files.txt",22050 * 4, sampling_rate=22050,augment=False)

    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2)
    test_loader = DataLoader(test_set, batch_size=1)

    ##########################
    # Dumping original audio #
    ##########################
    test_voc = []
    test_audio = []
    for i, x_t in enumerate(test_loader):
        # MRH:
        #x_t = x_t.cuda()
        x_t = x_t.to(device)
        s_t = fft(x_t).detach()

        test_voc.append(s_t.to(device))
        test_audio.append(x_t)

        audio = x_t.squeeze().cpu()
        save_sample(root / ("original_%d.wav" % i), 22050, audio)
        writer.add_audio("original/sample_%d.wav" % i, audio, 0, sample_rate=22050)

        if i == n_test_samples - 1:
            break

    costs = []
    start = time.time()

    # enable cudnn autotuner to speed up training
    torch.backends.cudnn.benchmark = True

    best_mel_reconst = 1000000
    steps = 0
    for epoch in range(1, epochs + 1):
        for iterno, x_t in enumerate(train_loader):
            x_t = x_t.to(device) # x_t size=  torch.Size([16, 1, 8192])
            s_t = fft(x_t).detach() # s_t size=  torch.Size([16, 80, 32])
            x_pred_t = netG(s_t.to(device)) # x_pred_t size()=  torch.Size([16, 1, 8192])

            with torch.no_grad():
                s_pred_t = fft(x_pred_t.detach())
                s_error = F.l1_loss(s_t, s_pred_t).item()

            #######################
            # Train Discriminator #
            #######################
            D_fake_det = netD(x_pred_t.to(device).detach())
            D_real = netD(x_t.to(device))

            loss_D = 0
            for scale in D_fake_det:
                loss_D += F.relu(1 + scale[-1]).mean()

            for scale in D_real:
                loss_D += F.relu(1 - scale[-1]).mean()

            netD.zero_grad()
            loss_D.backward()
            optD.step()

            ###################
            # Train Generator #
            ###################
            D_fake = netD(x_pred_t.to(device))

            loss_G = 0
            for scale in D_fake:
                loss_G += -scale[-1].mean()

            loss_feat = 0
            feat_weights = 4.0 / (n_layers_D + 1)
            D_weights = 1.0 / num_D
            wt = D_weights * feat_weights
            for i in range(num_D):
                for j in range(len(D_fake[i]) - 1):
                    loss_feat += wt * F.l1_loss(D_fake[i][j], D_real[i][j].detach())

            netG.zero_grad()
            (loss_G + lambda_feat * loss_feat).backward()
            optG.step()

            ######################
            # Update tensorboard #
            ######################
            costs.append([loss_D.item(), loss_G.item(), loss_feat.item(), s_error])

            writer.add_scalar("loss/discriminator", costs[-1][0], steps)
            writer.add_scalar("loss/generator", costs[-1][1], steps)
            writer.add_scalar("loss/feature_matching", costs[-1][2], steps)
            writer.add_scalar("loss/mel_reconstruction", costs[-1][3], steps)
            steps += 1

            if steps % save_interval == 0:
                st = time.time()
                with torch.no_grad():
                    for i, (voc, _) in enumerate(zip(test_voc, test_audio)):
                        pred_audio = netG(voc)
                        pred_audio = pred_audio.squeeze().cpu()
                        save_sample(root / ("generated_%d.wav" % i), 22050, pred_audio)
                        writer.add_audio(
                            "generated/sample_%d.wav" % i,
                            pred_audio,
                            epoch,
                            sample_rate=22050,
                        )

                torch.save(netG.state_dict(), root / "netG.pt")
                torch.save(optG.state_dict(), root / "optG.pt")

                torch.save(netD.state_dict(), root / "netD.pt")
                torch.save(optD.state_dict(), root / "optD.pt")

                if np.asarray(costs).mean(0)[-1] < best_mel_reconst:
                    best_mel_reconst = np.asarray(costs).mean(0)[-1]
                    torch.save(netD.state_dict(), root / "best_netD.pt")
                    torch.save(netG.state_dict(), root / "best_netG.pt")

                print("Took %5.4fs to generate samples" % (time.time() - st))
                print("-" * 100)

            if steps % log_interval == 0:
                print(
                    "Epoch {} | Iters {} / {} | ms/batch {:5.2f} | loss {}".format(
                        epoch,
                        iterno,
                        len(train_loader),
                        1000 * (time.time() - start) / log_interval,
                        np.asarray(costs).mean(0),
                    )
                )
                costs = []
                start = time.time()


if __name__ == "__main__":

    main()

### MRH: Playing files

In [ ]:
from IPython.display import display, Audio
display(Audio("/kaggle/working/MRH-Baseline/original_0.wav"))
display(Audio("/kaggle/working/MRH-Baseline/generated_0.wav"))